<a href="https://colab.research.google.com/github/leomercanti/Course_Advanced_Investing_with_AI/blob/main/Module_4_Automated_Trading_Systems_and_AI_Driven_Risk_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Module 4: Automated Trading Systems and AI-Driven Risk Management**

If you havent checked our previous modules yet, you can find them on the links below:
- [Module 1](https://colab.research.google.com/drive/15iRO6g-AyE2vGtdodh4xZ5RLmAcPcNV_)
- [Module 2](https://colab.research.google.com/drive/1Xr_athUWZH3iKZ6ubzvPnEAN9lPnee-1)
- [Module 3](https://colab.research.google.com/drive/1MwPrryj8q-DVMvuBhAOltWF5sdRVaBUq#scrollTo=WOPK7swfVHRb)


<br>

**Learning Goals:**
- Design and implement end-to-end automated trading systems that can execute trades without manual intervention.
- Combine supervised learning models (e.g., LSTMs for price prediction) with reinforcement learning agents to improve trading strategies.
- Apply AI-driven risk management techniques to minimize drawdowns and manage portfolio risk.
- Explore backtesting frameworks to evaluate and optimize trading strategies before deployment.

### 4.1 Core Readings and Resources

- **Textbook:** "Advances in Financial Machine Learning" by Marcos López de Prado
  - Chapter 8: Backtesting and Hyperparameter Optimization – Provides key insights into how to effectively backtest trading strategies and optimize machine learning models for finance.

- **Research Papers:**
  - “Artificial Intelligence in Algorithmic Trading” – Explores how AI technologies can be applied to build robust and adaptive trading systems.
  - “Risk Management and Portfolio Optimization Using AI” – A comprehensive paper discussing how AI techniques can be used for dynamic risk management and portfolio optimization.

- **Optional:**
  - “Learning to Trade with Deep Reinforcement Learning” – This paper delves into the integration of reinforcement learning for trading, highlighting several algorithms that can be employed.


### 4.2 Key Topics

**Building Automated Trading Systems**

- **Overview:** Automated trading systems (also called "algorithmic trading") allow traders to execute trades at a high frequency with reduced emotional bias. These systems make real-time decisions based on predefined rules or AI-driven insights, making them a perfect blend of machine learning and financial knowledge.

- **Main Components:**
  - **Signal Generation:** Using predictive models (e.g., LSTMs) or reinforcement learning agents to generate buy/sell signals.
  - **Order Execution:** Sending trade orders to the market through APIs like Interactive Brokers API or Alpaca API.
  - **Risk Management:** Using stop-loss, take-profit, and AI-based risk-adjusted strategies to protect the portfolio.

- **Hands-On Example:** Building a Simple Automated Trading System

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
# !pip install alpaca-trade-api
# import alpaca_trade_api as tradeapi

In [ ]:
# Download stock data
data = yf.download("MSFT", start="2016-01-01", end="2024-09-01")
close_prices = data['Close'].values

In [ ]:
# Quick data inspection
data.head()

In [ ]:
# Scale data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices.reshape(-1, 1))

In [ ]:
# Prepare data for LSTM
X_train, y_train = [], []
sequence_length = 60
for i in range(sequence_length, len(scaled_data)):
    X_train.append(scaled_data[i-sequence_length:i, 0])
    y_train.append(scaled_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
# Build LSTM model for signal generation
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))  # Predict next closing price
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Use Alpaca API to execute orders
api = tradeapi.REST('your-api-key', 'your-secret-key', base_url='https://paper-api.alpaca.markets')

In [ ]:
# Automated trading logic
def trade_stock():
    latest_price = yf.download("MSFT", period='1d')['Close'].iloc[-1]
    prediction = model.predict(np.array(latest_price).reshape(1, 60, 1))

    if prediction > latest_price:
        api.submit_order(symbol="MSFT", qty=10, side="buy", type="market", time_in_force="gtc")
    else:
        api.submit_order(symbol="MSFT", qty=10, side="sell", type="market", time_in_force="gtc")

In [ ]:
# Schedule trade execution (e.g., every hour)
import schedule
import time

schedule.every().hour.do(trade_stock)

while True:
    schedule.run_pending()
    time.sleep(1)

**AI-Driven Risk Management**

- **Overview:** Risk management is crucial in trading and portfolio optimization. AI-driven risk management leverages predictive models and reinforcement learning to dynamically adjust positions based on changing market conditions. This reduces exposure to large drawdowns and optimizes the risk-reward ratio of your portfolio.

- **Key Techniques:**
  - **Dynamic Position Sizing:** Adjusting the number of shares or contracts traded based on volatility and expected returns. Techniques like Kelly Criterion and risk parity are often used.
  - **Stop-Loss and Take-Profit Strategies:** Using machine learning models to set optimal stop-loss and take-profit levels dynamically.
  - **Value-at-Risk (VaR):** Applying machine learning to estimate the portfolio's risk, giving you a better understanding of potential losses.

- **Hands-On Example:** Implementing AI-Driven Risk Management

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Load historical data
data = yf.download("MSFT", start="2016-01-01", end="2024-09-01")

In [ ]:
# Calculate technical indicators
data['SMA'] = data['Close'].rolling(window=50).mean()
data['Volatility'] = data['Close'].rolling(window=50).std()

In [ ]:
# Prepare training data
X = data[['SMA', 'Volatility']].dropna().values
y = data['Close'][50:].values

In [ ]:
# Train Random Forest for risk prediction
model = RandomForestRegressor(n_estimators=100)
model.fit(X, y)

In [ ]:
# Predict future risk
latest_data = np.array([[data['SMA'].iloc[-1], data['Volatility'].iloc[-1]]])
predicted_risk = model.predict(latest_data)

print(f"Predicted risk (next day's volatility): {predicted_risk[0]:.2f}")

In [ ]:
# Define risk management rules based on predicted risk
max_risk = 0.05  # 5% maximum allowable risk
if predicted_risk[0] > max_risk:
    print("Reduce position size or exit trade.")
else:
    print("Safe to continue holding position.")

**Backtesting and Strategy Evaluation**

- **Overview:** Before deploying any trading strategy, it is essential to backtest it on historical data to evaluate performance. This helps you understand how the strategy would have performed in the past and optimize the parameters for future trading.

- **Key Concepts:**
  - **Sharpe Ratio:** Measures the risk-adjusted return of a portfolio.
  - **Max Drawdown:** The maximum loss from a peak to a trough in a portfolio’s value, indicating the risk level of the strategy.
  - **Alpha and Beta:** Performance indicators that measure how well a strategy performs relative to the market.

- **Hands-On Example:** Backtesting an AI Trading Strategy

In [ ]:
import backtrader as bt

In [ ]:
# Define a simple moving average strategy
class SMA_Strategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data.close, period=50)

    def next(self):
        if self.data.close > self.sma:
            self.buy(size=10)
        elif self.data.close < self.sma:
            self.sell(size=10)

In [ ]:
# Initialize backtesting environment
cerebro = bt.Cerebro()
data = bt.feeds.YahooFinanceData(dataname="MSFT", fromdate=pd.Timestamp("2016-01-01"), todate=pd.Timestamp("2024-09-01"))
cerebro.adddata(data)
cerebro.addstrategy(SMA_Strategy)

In [ ]:
# Run backtest
cerebro.run()

In [ ]:
# Evaluate performance metrics
final_portfolio_value = cerebro.broker.getvalue()
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")

### 4.3 Advanced Concepts: Integrating Reinforcement Learning with Automated Systems

**Combining Supervised and Reinforcement Learning:**

- **Hybrid Models:** Combining supervised models (e.g., LSTMs) for price prediction with reinforcement learning agents to manage portfolio allocation and risk.
- **Execution:** Using price predictions as a signal for reinforcement learning agents to take actions like buying or selling assets, optimizing overall performance.

**Risk-Aware RL:**

- **Risk-Constrained RL:** Implement risk constraints into reinforcement learning algorithms by defining utility functions that penalize high-risk actions.
- **Use Case:** A Deep Q-Learning agent that prioritizes low-volatility assets when market uncertainty increases.

### 4.4 End of Module Assignments and Practice (Optional)

- **Assignment 1:** Build an end-to-end automated trading system using reinforcement learning and backtest its performance on historical data. Incorporate risk management techniques like stop-loss and dynamic position sizing.

- **Assignment 2:** Implement AI-driven risk management by predicting the next day's portfolio risk using machine learning models and adjust your portfolio allocation accordingly.

By the end of this module, you have a solid understanding of how to build fully automated AI trading systems, combine reinforcement learning with predictive models, and apply AI techniques for dynamic risk management. You also gained experience in backtesting and evaluating strategies, setting the foundation for deploying AI-based systems in live financial markets.

This knowledge prepares you for the final module, where you’ll explore the deployment and scaling of AI systems in real-world trading environments.